In [1]:

import spacy
import pandas as pd
import json
import os
import re

from spacy.pipeline import EntityRuler
from spacy.lang.en import English

#nlp=English()
#arxikopoihsh tou nlp kai entity ruler
nlp=spacy.blank("en")
ruler=nlp.add_pipe("entity_ruler")



In [2]:
#in order to convert ubiai spacy ner 2 format into training input format

from  ecology_evolution import ecology
ecology[0][1]['entities'][1]

sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', ecology[0][0])

In [3]:
#in order to convert ubiai spacy ner 2 format into training input format
#in the end not used, spacy ner 2 format from ubiai seems to have wrong character position for entities annotated
from  dataset18 import d18
from  ecology_evolution import ecology

text = d18[0][0]
entities = d18[0][1]["entities"]
total_entities = len(entities)


sentences = [(sentence + '.', {'entities': []})
            for sentence in re.split(r'(?<=[A-Za-z0-9])\. +(?=[A-Z])', text)]  #split text in . and next Uppercase


# text.split(".")
total_len = 0
i = 0
i_entity = 0
for sentence in sentences:
    sentence_len = len(sentence[0])
    while (total_entities > i_entity and total_len+sentence_len > entities[i_entity][0]):
        sentences[i][1]['entities'].append(
            (entities[i_entity][0]-total_len, entities[i_entity][1]-total_len, entities[i_entity][2]))
        i_entity += 1

    total_len += sentence_len
    i += 1

a=sentences
a


[('710  Enrico Tortonese on yome  longer than broad, 5th to 10th transverse, gradually  increasing in width, the penultimate about twice as broad  as long.',
  {'entities': [(96, 102, 'BODY_SIZE')]}),
 ('Thorax broader than the head, as long as broad,  the sides feebly rounded in front, straighter and a little  retracted behind, rather more distinctly punctured than  the head, the ground-sculpture similars Elytra  long as the thorax and at the base slightly narrower,  apex a little broader than the thorax and there broader  than long, the puncturation very similar, fuely  Abdomen longer than the fore-parts, very finely,  moderately closely punctured And with very  less transverse striate ground-sculpture  BRARY  3. 8th tergite with very small ar in  middle of posterior margin, externally with a very small  and obscure denticle on each side.',
  {'entities': []}),
 ('Kings Domain, Westport, Westland.', {'entities': []}),
 ('Type in Brookes  Collection, co-type in mine.', {'entities': []

In [5]:
#in order to convert ubiai iob format into spacy input for training
#gia na to diavasei swsta esvhsa thn prwth grammh kai evala no header
import re
import pandas as pd


df = pd.read_csv("dataset3.tsv", sep='\t+', header=None,
                 skiprows=1, engine='python')


def iob_to_spacy(df):
    words = df[0]
    tags = df[1]
    total_words = len(words)
    data = []
    data_i = 0
    i = 0


    sentence = ""
    sentence_len = 0
    sentence_entities = []
    for word in words:
        dot_index = word.find('.')
        if (dot_index != -1):
            # vrethike sth thesh dot_index
            if (dot_index == len(word)-1):
                # einai telefteo gramma
                # prepei na checkarw to 1o gramma sto epomeno word
                if (total_words-1 > i):
                    # uparxei to epomeno word
                    # elegxw to 1o gramma an einai kefaleo
                    if (words[i+1][0].isupper()):
                        sentence += word
                        if (tags[i] != 'O'):
                            entity = (sentence_len, len(sentence), tags[i])
                            sentence_entities.append(entity)
                        sentence_len = len(sentence)
                        new_val = (sentence, {"entities": sentence_entities})
                        data.append(new_val)
                        sentence = ""
                        sentence_len = 0
                        sentence_entities = []
                    else:
                        sentence += word + " "
                        if (tags[i] != 'O'):
                            entity = (sentence_len, len(sentence), tags[i])
                            sentence_entities.append(entity)
                        sentence_len = len(sentence)
        else:
            sentence += word + " "
            if (tags[i] != 'O'):
                entity = (sentence_len, len(sentence), tags[i])
                sentence_entities.append(entity)
            sentence_len = len(sentence)
        i += 1
    return data

dataset3spacy=iob_to_spacy(df)
# print(dataset3spacy)

In [6]:
# sent = dataset3spacy[10]
# num_of_ents = len(entities)

for sent in dataset3spacy:
    entities = sent[1]['entities']
    new_entities = []
    entity_start = 0
    entity_end = 0
    entity_type = ""
    for entity in entities:
        if entity[2][0] == "B":
            if entity_end != 0:
                new_entity = (entity_start, entity_end, entity_type)
                new_entities.append(new_entity)
            entity_start = entity[0]
            entity_end = entity[1]
            entity_type = entity[2]
        else:
            entity_end = entity[1]
    if entity_end != 0:
        new_entity = (entity_start, entity_end, entity_type)
        new_entities.append(new_entity)
        sent[1]['entities']=new_entities
    print(sent)

('OF COMMERCE AND INDUSTRY, EGYPT HYDROBIOLOGY AND FISHERIES DIRECTORATE NOTES AND MEMOIRS No. 29 A CHEMICAL STUDY OF THE EGYPTIAN SARDINELLA IN THE FAT CONTENT OF WHOLE FISH, FLESH, AND GONADS BY M.', {'entities': []})
('K.', {'entities': []})
('EL SABY, (Liverpool) Scientific Assistant, Hydrobiology and Fisheries Directorate, Alexandria CAIRO GOVERNMENT PRESS, BULAQ 1937 MINISTRY OF COMMERCE AND INDUSTRY A Chemical Study of the Egyptian Sardinella -Variation in the fat content of whole fish, flesh, and gonads BY M.', {'entities': []})
('K.', {'entities': []})
('EL SABY, (Liverpool) Scientific Assistant, Hydrobiology and Fisheries Directorate, Alexandria INTRODUCTION Save what is recorded in the Fisheries reports issued since 1920 by the Fisheries Research of Egypt, there is no published account concerned with a description of the Egyptian Sardine Fishery, and for this reason it appears to me to be desirable to write a short note on this fishery as regards composition, locality, seaso

In [4]:
df=pd.read_csv("dictionaries-annot\Distribution_descriptors.csv",header=None)
df1=pd.read_csv("dictionaries-annot\Life_stages.csv",header=None)
df2=pd.read_csv("dictionaries-annot\Body_size.csv",header=None)
#df3=pd.read_csv("dictionaries\biological_descriptors.csv",header=None,encoding='cp1252')

df_distr_descr=df.iloc[:,0]
df_life_stages=df1.iloc[:,0]
df_body_size=df2.iloc[:,0]
#df_biol_descr=df3.iloc[:,0]
#to lige stages ousiastika einai mesa sta biological descriptors



In [5]:
import random
import spacy.cli
from spacy.tokens import DocBin
from tqdm import tqdm

In [6]:
#annotated corpus data for training
TRAIN_DATA = [ ("interations between infaunal species which can attain high densities within cages.",{"entities":[(20,29,"Distribution_descriptor"),(76,81,"Distribution_descriptor")]}),
("infauna, they also provide a good opportunity to study competitive or adult-larval ",{"entities":[(0,7,"Distribution_descriptor"),(70,75,"Life_stage"),(76,82,"Life_stage"),(70,82,"Life_stage")]}),
(" on ",{"entities":[]}),
("influence of larger predators such as fish, crabs, and epibenthic predatory benthos on",{"entities":[(76,84,"Distribution_descriptor"),(55,66,"Distribution_descriptor")]}),
("with those in control plots. Although cages are most commonly used to assess the ",{"entities":[]}),
("or in which densities of such adults are varied, and to compare the larval densities there ",{"entities":[(30,36,"Life_stage"),(68,75,"Life_stage")]}),
("establish plots from which adults which may interact with settling larvae are removed ",{"entities":[(27,33,"Life_stage"),(67,73,"Life_stage")]}),
("The basic procedure in field experiments examining adult-larval interactions is to ",{"entities":[(51,56,"Life_stage"),(57,63,"Life_stage"),(51,63,"Life_stage")]}),
("and some islands in the vicinity of the latter.",{"entities":[]}),
("known from the rivers of Australia, New Guinea ",{"entities":[(15,22,"Distribution_descriptor")]}),
("marine ancestors and until recently have only been ",{"entities":[(0,7,"Distribution_descriptor")]}),
("All freshwater atherinid species are derived from ",{"entities":[(4,15,"Distribution_descriptor")]}),
("from Tumaco, Colombia, according to Dyer. ",{"entities":[]}),
("Southeast Pacific, the family is only known only ",{"entities":[]}),
("Caribbean) known from the New World. In the ",{"entities":[]}),
("(predominantly in the shore waters of the ",{"entities":[(22,34,"Distribution_descriptor")]}),
("Atherinomorus stipes and Hypoatherina harringtonensis ",{"entities":[]}),
("World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, ",{"entities":[(39,51,"Distribution_descriptor"),(17,37,"Distribution_descriptor"),(68,74,"Distribution_descriptor")]}),
("be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old ",{"entities":[(50,59,"Body_size"),(57,59,"Body_size"),(50,56,"Body_size")]}),
("Atherinids are small marine, estuarine and freshwater fishes not exceeding 120 mm SL (a soon to ",{"entities":[(21,27,"Distribution_descriptor"),(29,39,"Distribution_descriptor"),(43,54,"Distribution_descriptor"),(82,84,"Body_size"),(75,81,"Body_size")]}),
("lagoon population but a potential contribution to the genetic flux among coastal populations. ",{"entities":[(0,7,"Distribution_descriptor"),(73,81,"Distribution_descriptor")]}),
("the estuarine larvae that drifted out of the lagoon seemed to survive, suggesting a loss of individuals from the ",{"entities":[(4,14,"Distribution_descriptor"),(14,21,"Life_stage"),(45,52,"Distribution_descriptor")]}),
("CW to LR, possibly reducing their survival. Generally, eggs were more vulnerable to drift than larvae. Many of ",{"entities":[(0,2,"Distribution_descriptor"),(6,8,"Distribution_descriptor"),(55,60,"Life_stage"),(95,101,"Life_stage")]}),
("when heavy rains led to severe lagoon discharges. Similarly, unusual strong onshore winds advected eggs from ",{"entities":[(5,16,"Distribution_descriptor"),(31,37,"Distribution_descriptor"),(76,89,"Distribution_descriptor"),(99,104,"Life_stage")]}),
("larvae to LR. Eggs and larvae of estuarine species that spawned within LR were effectively retained there, except ",{"entities":[(0,6,"Life_stage"),(13,18,"Life_stage"),(22,29,"Life_stage"),(32,42,"Distribution_descriptor"),(70,73,"Distribution_descriptor")]}),
("transport of these larvae into LR. Conversely, there was no evidence of entrance of marine estuarine-independent ",{"entities":[(19,25,"Life_stage"),(31,33,"Distribution_descriptor"),(91,100,"Distribution_descriptor"),(91,112,"Distribution_descriptor"),(84,90,"Distribution_descriptor")]}),
("and light onshore winds during the open-sandbar period were the most important process that promoted the ",{"entities":[(40,47,"Distribution_descriptor"),(10,23,"Distribution_descriptor")]}),
("abundance in CW. Sandbar closure did not impede the entrance of these species. Overwashing events (closed bar) ",{"entities":[(13,15,"Distribution_descriptor"),(17,24,"Distribution_descriptor")]}),
("runoff) and species-specific traits (e.g. life cycle strategy, spawning season) determined ichthyoplankton exchange vs. retention. Marine estuarine-related species entered LR mostly during summer-autumn, at their peak ",{"entities":[(0,6,"Distribution_descriptor"),(91,106,"Life_stage"),(131,137,"Distribution_descriptor"),(138,147,"Distribution_descriptor"),(138,155,"Distribution_descriptor"),(172,174,"Distribution_descriptor")]}),
("nursery ground. Larval stages of six marine species were found at CW. Environmental factors (e.g. wind and ",{"entities":[(16,23,"Life_stage"),(36,43,"Distribution_descriptor"),(64,66,"Distribution_descriptor"),(96,100,"Distribution_descriptor")]}),
("exchange of water and ichthyoplankton. At least three estuarine species were found to spawn and use LR as a ",{"entities":[(12,17,"Distribution_descriptor"),(22,37,"Life_stage"),(54,64,"Distribution_descriptor"),(100,102,"Distribution_descriptor")]}),
("conditions and ichthyoplankton communities between LR and CW, and also evidenced different mechanisms of ",{"entities":[(51,54,"Distribution_descriptor"),(57,59,"Distribution_descriptor")]}),
("along a year in LR and CW to capture different sandbar states (open/closed) and lagoon-sea connectivity processes (e.g. lagoon discharges, marine intrusion, overwashing). Results showed differences in environmental ",{"entities":[(16,18,"Distribution_descriptor"),(23,25,"Distribution_descriptor"),(47,55,"Distribution_descriptor"),(80,90,"Distribution_descriptor"),(119,126,"Distribution_descriptor"),(137,144,"Distribution_descriptor")]}),
("waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys ",{"entities":[(0,7,"Distribution_descriptor"),(8,10,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal ",{"entities":[(84,86,"Body_size"),(76,83,"Distribution_descriptor"),(101,108,"Distribution_descriptor"),(92,108,"Distribution_descriptor"),(64,82,"Distribution_descriptor"),(84,86,"Distribution_descriptor")]}),
("spawned previously. A summary of the life-cycle is also ",{"entities":[]}),
("indicated that some females in this sample had ",{"entities":[]}),
("(8.8%). A few atretic oocytes and empty follicles ",{"entities":[]}),
("stage (91.2%) or in the rst stage of vitellogenesis ",{"entities":[(37,51,"Body_size")]}),
("and spermatozoa. Oocytes were in the previtellogenic ",{"entities":[]}),
("sample were in a resting stage. Testes lacked spermatids ",{"entities":[]}),
("histology indicated that the gonads of this November ",{"entities":[]}),
("Although specimens were large and sexually mature, the ",{"entities":[]}),
("index (IG) was 0.9% for females and 0.2% for males. ",{"entities":[]}),
("1.6% for females and 1.7% for males; gonadosomatic ",{"entities":[]}),
("relationship to W and LT. Hepatosomatic index (IH) was ",{"entities":[(16,17,"Body_size"),(22,24,"Body_size")]}),
("showed no difference due to sex, but a signicant weak ",{"entities":[]}),
("Subcutaneous lipid thickness averaged 2.6 mm and ",{"entities":[(19,29,"Body_size"),(38,44,"Body_size")]}),
("26.7 kg (range 10.460.3 kg). The length-weight relationship was W=3.44 105 L2:85T (n 28, r2 0.96). ",{"entities":[(0,7,"Body_size"),(9,26,"Body_size"),(0,27,"Body_size"),(33,46,"Body_size"),(33,39,"Body_size"),(40,46,"Body_size"),(65,71,"Body_size"),(74,84,"Body_size")]}),
("in McMurdo Sound, Antarctica. Total length (LT) averaged 127.3 cm (range 90162 cm) and weight (W) was ",{"entities":[(30,42,"Body_size"),(44,46,"Body_size"),(30,47,"Body_size"),(57,65,"Body_size"),(67,81,"Body_size"),(87,97,"Body_size"),(87,93,"Body_size"),(94,97,"Body_size")]}),
("capture on a set line near the southern limit of its range ",{"entities":[]}),
("Twenty-eight specimens of the large not othenioid Dissostichus mawsoni were dissected after ",{"entities":[]}),
("species.",{"entities":[]}),
("appears to be influenced by conflicting selection pressures that may differ between the ",{"entities":[]}),
("R. whero and competition for habitat in both species. Body size in the Ruanoho species ",{"entities":[(29,37,"Distribution_descriptor"),(54,63,"Body_size")]}),
("size differences in the Ruanoho pair are consistent with female choice for smaller males in ",{"entities":[(0,5,"Body_size")]}),
("while female R. decemdigitatus had no preference for male size. Results suggest that body ",{"entities":[(58,62,"Body_size")]}),
("in heterospecific trials. Female R. where showed a strong preference for smaller males, ",{"entities":[]}),
("a subset of that displayed by R. decemdigitatus, while no mating behaviour was observed ",{"entities":[(58,64,"Distribution_descriptor")]}),
("R. whero were displaced into less preferred habitats. Courtship behaviour in R. whero was ",{"entities":[(44,52,"Distribution_descriptor")]}),
("dominant in inter- and intraspecific contests for the preferred rock habitat, while small ",{"entities":[(64,76,"Distribution_descriptor")]}),
("trials. Both species exhibited similar habitat preferences, but large R. decemdigitatus were ",{"entities":[(39,47,"Distribution_descriptor"),(39,58,"Distribution_descriptor"),(64,70,"Body_size")]}),
("setting. Reproductive behaviour was assessed in courtship, mate choice and hybridisation ",{"entities":[(9,31,"Distribution_descriptor"),(48,57,"Distribution_descriptor"),(75,88,"Distribution_descriptor")]}),
("trials based on the field data to test habitat preference and competitive ability in a common ",{"entities":[(39,47,"Distribution_descriptor")]}),
("substratum. The effect of body size on substratum use was investigated using laboratory ",{"entities":[(0,10,"Distribution_descriptor"),(26,35,"Body_size"),(39,50,"Distribution_descriptor")]}),
("20 m. In sites where the species co-occurred, R. whero was less associated with rock ",{"entities":[(0,4,"Body_size"),(80,84,"Distribution_descriptor")]}),
("while R. whero was frequently found in sheltered to moderately exposed areas down to ",{"entities":[]}),
("distributions and differ in body size, attaining a total length of 12 cm and 9 cm, respectively. R. decemdigitatus was most commonly found in sheltered areas shallower than 5 m, ",{"entities":[(28,37,"Body_size"),(51,63,"Body_size"),(67,72,"Body_size"),(77,81,"Body_size"),(51,72,"Body_size"),(51,81,"Body_size"),(142,157,"Distribution_descriptor")]}),
("New Zealand. Ruanoho decemdigitatus and Ruanoho whero have overlapping sympatric ",{"entities":[]}),
("The effect of body size on spatial resource competition and reproductive isolation was examined in a sister species pair of subtidal triplefin fishes (F. Tripterygiidae) in ",{"entities":[(14,23,"Body_size")]}),
("fin height and eye size, which are invariably greater in P. muelleri. ",{"entities":[(4,11,"Body_size"),(15,23,"Body_size")]}),
("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[(7,24,"Distribution_descriptor")]}),
("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor"),(5,12,"Distribution_descriptor")]}),
("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(64,66,"Body_size"),(57,66,"Body_size")]}),
("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]})]

# #annotaded paper data
# import json
# with open('dictionaries-annot/annotations_paper1.json', 'r') as f:
#     data = json.load(f)
    
# TRAIN_DATA_paper = data['annotations']


In [7]:
#ksana annotate to corpus +1-2 abstracts
train_corp=[("Pairwise comparisons of sister families do not reveal any trends between familial diversity and any of the life history traits. The great variability of life history traits within polychaete families may explain the lack of significant results; perhaps no trends are seen because polychaete life history traits cannot be generalized at the family level.",{"entities":[]}),
("or with planktotrophic larval development. Each of these factors has been advanced as cause for high diversity in other taxonomic assemblages. Here, we establish that the diversification pattern of polychaete families is non-random, but the data collected show no significant correlations between familial diversity and several life history traits including age at first reproduction, life span, body size, fecundity, and egg size. ",{"entities":[(23,29,"Life_stage")]}),
("Polychaetes comprise 9000 described species classified in over 70 families and exhibit a great variety of life history strategies. Our goal was to test whether diverse polychaete families are characterized by species with short generation times, high reproductive output, small body size, ",{"entities":[]}),
("Within many phylogenetic assemblages, a pattern of domination has been observed: one or a few clades have had many more speciation events or fewer extinctions than other clades in a particular assemblage. We investigated this phenomenon in the polychaete annelids. ",{"entities":[]}),
("Amphipoda, and Decapoda) and Polychaeta ",{"entities":[]}),
("color and size of fins. Similar to other representatives of the genus Hemilepidotus, H. zapus is a benthoichthyophage with a wide food range consisting mainly of bottom and demersal crustaceans (Cirripedia, ",{"entities":[]}),
("is relatively low (on average 12800 eggs); there is dimorphism in size (females are larger than males) and in ",{"entities":[(36,40,"Life_stage")]}),
("330 g, and age reaches nine years. It spawns in August to September; the spawning is single-batch; fecundity ",{"entities":[(0,5,"Body_size")]}),
("the most short-cycled representative of the genus Hemilepidotus, whose maximum size reaches 26 cm and ",{"entities":[(71,83,"Body_size"),(92,97,"Body_size")]}),
("in the southern part (on the slopes of the underwater ridge of the outer Kurils). This species is the smallest and ",{"entities":[]}),
("the spatial-bathymetric distribution of, zapus in Pacific waters off the Northern Kurils (from 4750' to 5030' N, depths of 100-850 m) is analyzed, its thermal environment is characterized. In the northern part of the investigated area near Paramushir Island, H. zapus is represented by larger specimens staying at greater depths than ",{"entities":[(159,170,"Distribution_descriptor")]}),
("Northern Kurils from Paramushir Island to Ketoy Island. Taking into consideration trawl catches of 1996-2001, ",{"entities":[]}),
("used in the key for reliable identification of species of both genera irrespective of localities, sex, age, and condition of a specimen. In Russian waters, Hemilepidotus zapus is distributed off the Commander Islands and the ",{"entities":[(149,155,"Distribution_descriptor")]}),
("revealed. Additional taxonomic features are found. In combination with some previously used features, they are ",{"entities":[]}),
("features for the determination of taxonomical boundaries of species belonging to this group. Improved diagnoses of species are composed. Significant transgression of most features previously used in identification is ",{"entities":[]}),
("The Comparative morphological description of 248 specimens of six species of the genera Hemilepidotus and Melletes served as the basis for the assessment of the variation of over 28 external morphological ",{"entities":[]}),
("fin height and eye size, which are invariably greater in P. muelleri. ",{"entities":[(15,23,"Body_size"),(0,10,"Body_size")]}),
("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[]}),
("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor")]}),
("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(54,63,"Body_size"),(64,66,"Body_size")]}),
("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]}),
("The marine catfish genus Paraplotosus Bleeker is reviewed. Three ",{"entities":[(4,10,"Distribution_descriptor")]}),
("species.",{"entities":[]}),
("appears to be influenced by conflicting selection pressures that may differ between the ",{"entities":[]}),
("R. whero and competition for habitat in both species. Body size in the Ruanoho species ",{"entities":[(29,36,"Distribution_descriptor"),(54,63,"Body_size")]}),
("size differences in the Ruanoho pair are consistent with female choice for smaller males in ",{"entities":[(0,5,"Body_size")]}),
("while female R. decemdigitatus had no preference for male size. Results suggest that body ",{"entities":[(85,89,"Body_size")]}),
("in heterospecific trials. Female R. whero showed a strong preference for smaller males, ",{"entities":[]}),
("a subset of that displayed by R. decemdigitatus, while no mating behaviour was observed ",{"entities":[(55,74,"Distribution_descriptor"),(55,64,"Distribution_descriptor")]}),
("R. whero were displaced into less preferred habitats. Courtship behaviour in R. whero was ",{"entities":[(44,52,"Distribution_descriptor"),(64,73,"Distribution_descriptor")]}),
("dominant in inter- and intraspecific contests for the preferred rock habitat, while small ",{"entities":[(64,76,"Distribution_descriptor"),(64,68,"Distribution_descriptor")]}),
("trials. Both species exhibited similar habitat preferences, but large R. decemdigitatus were ",{"entities":[(39,46,"Distribution_descriptor")]}),
("setting. Reproductive behaviour was assessed in courtship, mate choice and hybridisation ",{"entities":[(9,31,"Distribution_descriptor")]}),
("trials based on the field data to test habitat preference and competitive ability in a common ",{"entities":[(39,46,"Distribution_descriptor")]}),
("substratum. The effect of body size on substratum use was investigated using laboratory ",{"entities":[(0,10,"Distribution_descriptor"),(26,35,"Body_size"),(39,49,"Distribution_descriptor")]}),
("20 m. In sites where the species co-occurred, R. whero was less associated with rock ",{"entities":[(0,4,"Distribution_descriptor")]}),
("while R. whero was frequently found in sheltered to moderately exposed areas down to ",{"entities":[(39,48,"Distribution_descriptor"),(52,70,"Distribution_descriptor")]}),
("distributions and differ in body size, attaining a total length of 12 cm and 9 cm, respectively. R. decemdigitatus was most commonly found in sheltered areas shallower than 5 m, ",{"entities":[(28,37,"Body_size"),(51,63,"Body_size"),(67,81,"Body_size"),(67,72,"Body_size"),(77,81,"Body_size"),(143,158,"Distribution_descriptor"),(159,177,"Distribution_descriptor")]}),
("New Zealand. Ruanoho decemdigitatus and Ruanoho whero have overlapping sympatric ",{"entities":[]}),
("The effect of body size on spatial resource competition and reproductive isolation was examined in a sister species pair of subtidal triplefin fishes (F. Tripterygiidae) in ",{"entities":[(14,23,"Body_size")]}),
("spawned previously. A summary of the life-cycle is also ",{"entities":[]}),
("indicated that some females in this sample had ",{"entities":[]}),
("(8.8%). A few atretic oocytes and empty follicles ",{"entities":[]}),
("stage (91.2%) or in the rst stage of vitellogenesis ",{"entities":[]}),
("and spermatozoa. Oocytes were in the previtellogenic ",{"entities":[]}),
("sample were in a resting stage. Testes lacked spermatids ",{"entities":[]}),
("histology indicated that the gonads of this November ",{"entities":[]}),
("Although specimens were large and sexually mature, the ",{"entities":[]}),
("index (IG) was 0.9% for females and 0.2% for males. ",{"entities":[]}),
("1.6% for females and 1.7% for males; gonadosomatic ",{"entities":[]}),
("relationship to W and LT. Hepatosomatic index (IH) was ",{"entities":[(16,17,"Body_size"),(22,24,"Body_size")]}),
("showed no dierence due to sex, but a signicant weak ",{"entities":[]}),
("Subcutaneous lipid thickness averaged 2.6 mm and ",{"entities":[(13,28,"Body_size"),(38,44,"Body_size")]}),
("26.7 kg (range 10.460.3 kg). The length-weight relationship was W=3.44  105 L2:85T (n 28, r2 0.96). ",{"entities":[(0,27,"Body_size"),(0,7,"Body_size")]}),
("in McMurdo Sound, Antarctica. Total length (LT) averaged 127.3 cm (range 90162 cm) and weight (W) was ",{"entities":[(30,42,"Body_size"),(44,46,"Body_size"),(30,47,"Body_size"),(57,65,"Body_size"),(67,81,"Body_size"),(57,82,"Body_size"),(87,97,"Body_size"),(87,93,"Body_size")]}),
("capture on a set line near the southern limit of its range ",{"entities":[]}),
("Twenty-eight specimens of the large notothenioid Dissostichus mawsoni were dissected after ",{"entities":[]}),
("ecological traits that are body size and trophic position.",{"entities":[(27,36,"Body_size")]}),
("predicting species loss effect on functioning based on easily measured ",{"entities":[]}),
("ecosystem functioning than random losses. Our results suggest apparent ease in ",{"entities":[]}),
("multitrophic systems, a more negative impact of ordered extinction sequences on ",{"entities":[(0,12,"Distribution_descriptor")]}),
("loss effects on functioning. Our study is the first to show experimentally, in ",{"entities":[]}),
("ordered by body size, random extinction sequences had lower multiple species ",{"entities":[(11,20,"Body_size")]}),
("effect of body size within trophic position levels. Compared to extinctions ",{"entities":[(10,19,"Body_size")]}),
("body size on functioning. The effect was much stronger when considering the ",{"entities":[(0,9,"Body_size")]}),
("and freshwater multitrophic systems. We found a significant, but weak, effect of ",{"entities":[(4,14,"Distribution_descriptor")]}),
("traits. Here, we examined the predictive capacity of species body size in marine ",{"entities":[(61,70,"Body_size"),(74,80,"Distribution_descriptor")]}),
("indicator because of its relationship to extinction risk and several functional ",{"entities":[]}),
("species loss on ecosystem functioning. Body size is often considered as a good ",{"entities":[(39,48,"Body_size")]}),
("There is an urgent need to develop predictive indicators of the effect of ",{"entities":[]}),
("lagoon population but a potential contribution to the genetic flux among coastal populations.",{"entities":[(0,6,"Distribution_descriptor"),(73,80,"Distribution_descriptor")]}),
("the estuarine larvae that drifted out of the lagoon seemed to survive, suggesting a loss of individuals from the ",{"entities":[(4,13,"Distribution_descriptor"),(14,20,"Life_stage"),(45,51,"Distribution_descriptor")]}),
("CW to LR, possibly reducing their survival. Generally, eggs were more vulnerable to drift than larvae. Many of ",{"entities":[(0,2,"Distribution_descriptor"),(6,8,"Distribution_descriptor"),(55,59,"Life_stage"),(95,101,"Life_stage")]}),
("when heavy rains led to severe lagoon discharges. Similarly, unusual strong onshore winds advected eggs from ",{"entities":[(31,37,"Distribution_descriptor"),(99,103,"Life_stage"),(69,89,"Distribution_descriptor")]}),
("larvae to LR. Eggs and larvae of estuarine species that spawned within LR were effectively retained there, except ",{"entities":[(0,6,"Life_stage"),(10,12,"Distribution_descriptor"),(14,18,"Life_stage"),(23,29,"Life_stage"),(33,42,"Distribution_descriptor"),(71,73,"Distribution_descriptor")]}),
("transport of these larvae into LR. Conversely, there was no evidence of entrance of marine estuarine-independent ",{"entities":[(19,25,"Life_stage"),(31,33,"Distribution_descriptor"),(84,90,"Distribution_descriptor"),(91,100,"Distribution_descriptor")]}),
("and light onshore winds during the open-sandbar period were the most important process that promoted the ",{"entities":[(10,23,"Distribution_descriptor")]}),
("abundance in CW. Sandbar closure did not impede the entrance of these species. Overwashing events (closed bar) ",{"entities":[(13,15,"Distribution_descriptor"),(17,24,"Distribution_descriptor"),(79,91,"Distribution_descriptor")]}),
("runoff) and species-specific traits (e.g. life cycle strategy, spawning season) determined ichthyoplankton exchange vs. retention. Marine estuarine-related species entered LR mostly during summer-autumn, at their peak ",{"entities":[(0,6,"Distribution_descriptor"),(91,106,"Distribution_descriptor"),(132,138,"Distribution_descriptor"),(139,148,"Distribution_descriptor"),(173,175,"Distribution_descriptor")]}),
("nursery ground. Larval stages of six marine species were found at CW. Environmental factors (e.g. wind and ",{"entities":[(16,22,"Life_stage"),(37,43,"Distribution_descriptor"),(66,68,"Distribution_descriptor"),(70,83,"Distribution_descriptor"),(98,102,"Distribution_descriptor")]}),
("exchange of water and ichthyoplankton. At least three estuarine species were found to spawn and use LR as a ",{"entities":[(12,17,"Distribution_descriptor"),(22,37,"Distribution_descriptor"),(54,63,"Distribution_descriptor"),(100,102,"Distribution_descriptor")]}),
("conditions and ichthyoplankton communities between LR and CW, and also evidenced different mechanisms of ",{"entities":[(51,53,"Distribution_descriptor"),(58,60,"Distribution_descriptor")]}),
("along a year in LR and CW to capture different sandbar states (open/closed) and lagoon-sea connectivity processes (e.g. lagoon discharges, marine intrusion, overwashing). Results showed differences in environmental ",{"entities":[(16,18,"Distribution_descriptor"),(23,25,"Distribution_descriptor"),(47,54,"Distribution_descriptor"),(63,74,"Distribution_descriptor"),(80,86,"Distribution_descriptor"),(87,90,"Distribution_descriptor"),(121,127,"Distribution_descriptor"),(140,146,"Distribution_descriptor"),(158,169,"Distribution_descriptor")]}),
("waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys ",{"entities":[(0,6,"Distribution_descriptor"),(8,10,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal ",{"entities":[(64,75,"Distribution_descriptor"),(76,82,"Distribution_descriptor"),(84,86,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and ",{"entities":[]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys",{"entities":[(64,87,"Distribution_descriptor"),(76,82,"Distribution_descriptor"),(101,108,"Distribution_descriptor")]}),
("and some islands in the vicinity of the latter. ",{"entities":[]}),
("known from the rivers of Australia, New Guinea ",{"entities":[(15,21,"Distribution_descriptor")]}),
("marine ancestors and until recently have only been ",{"entities":[(0,6,"Distribution_descriptor")]}),
("All freshwater atherinid species are derived from ",{"entities":[(4,14,"Distribution_descriptor")]}),
("from Tumaco, Colombia, according to Dyer. ",{"entities":[]}),
("Southeast Pacific, the family is only known only ",{"entities":[]}),
("Caribbean) known from the New World. In the ",{"entities":[]}),
("(predominantly in the shore waters of the ",{"entities":[(22,34,"Distribution_descriptor")]}),
("Atherinomorus stipes and Hypoatherina harringtonensis ",{"entities":[]}),
("World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, ",{"entities":[(17,37,"Distribution_descriptor"),(39,50,"Distribution_descriptor"),(68,74,"Distribution_descriptor")]}),
("be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old ",{"entities":[(50,59,"Body_size"),(50,56,"Body_size"),(57,59,"Body_size")]}),
("Atherinids are small marine, estuarine and freshwater fishes not exceeding 120 mm SL (a soon to ",{"entities":[(21,27,"Distribution_descriptor"),(29,38,"Distribution_descriptor"),(43,53,"Distribution_descriptor"),(15,20,"Body_size"),(75,84,"Body_size"),(82,84,"Body_size"),(75,81,"Body_size")]}),
("fauna which invaded the freshwaters of New Caledonia.",{"entities":[(0,5,"Distribution_descriptor"),(24,35,"Distribution_descriptor")]}),
("Palaeocene, must have led to a divergence of the ancestral ",{"entities":[]}),
("Australia and its emergence as a separate island, post ",{"entities":[]}),
("events, which led to the separation of New Caledonia from ",{"entities":[]}),
("marine environment i.e. Arafura Sea. The zoogeographic ",{"entities":[(0,6,"Distribution_descriptor"),(24,35,"Distribution_descriptor")]}),
("from a common ancestor likely to have occurred in a ",{"entities":[]}),
("coastal and marine fishes, having probably been derived ",{"entities":[(0,7,"Distribution_descriptor"),(12,18,"Distribution_descriptor")]}),
("new species must have commonality with the Australian ",{"entities":[]}),
("merits a subfamilial status. The evolutionary history of this ",{"entities":[]}),
("known atherinids. Bleheratherina pierucciae can be immediately recognised by the unusual structure of its mouthparts. Other distinct osteological characters confirm that it ",{"entities":[]}),
("list since the new species is distinct and different from all ",{"entities":[]}),
("fourth subfamily, Bleheratherininae, is now added to this ",{"entities":[]}),
("into three subfamilies has been briefly reviewed and a ",{"entities":[]}),
("Leptatherina and Stenatherina) and an atherionid (Atherion). Dyer & Chernoffs (1996) division of Atherinidae ",{"entities":[]}),
("Atherion, Craterocephalus, Hypoatherina, Kestratherina, ",{"entities":[]}),
("(2656.9S 16614E) and Pirogues Rivers, New Caledonia. The new species has been compared with other IndoPacific atherinids, both freshwater and marine (representatives of genera Atherinason, Atherinomorus, Atherinosoma, ",{"entities":[(30,36,"Distribution_descriptor"),(129,139,"Distribution_descriptor"),(144,150,"Distribution_descriptor")]}),
("Bleheratherina pierucciae is described from Tontouta ",{"entities":[]})]# 
# ("interations between infaunal species which can attain high densities within cages.",{"entities":[(20,28,"Distribution_descriptor")]}),
# ("infauna, they also provide a good opportunity to study competitive or adult-larval ",{"entities":[(0,7,"Distribution_descriptor"),(70,82,"Life_stage"),(70,75,"Life_stage"),(76,82,"Life_stage")]}),
# ("influence of larger predators such as fish, crabs, and epibenthic predatory benthos on ",{"entities":[(55,65,"Distribution_descriptor")]}),
# ("Although cages are most commonly used to assess the ",{"entities":[]}),
# ("with those in control plots.",{"entities":[]}),
# ("or in which densities of such adults are varied, and to compare the larval densities there ",{"entities":[(30,36,"Life_stage"),(68,74,"Life_stage")]}),
# ("establish plots from which adults which may interact with settling larvae are removed ",{"entities":[(27,33,"Life_stage"),(67,73,"Life_stage")]}),
# ("The basic procedure in field experiments examining adult-larval interactions is to ",{"entities":[(51,63,"Life_stage")]}),
# ("fin height and eye size, which are invariably greater in P. muelleri.",{"entities":[(0,10,"Body_size"),(15,23,"Body_size")]}),
# ("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
# ("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[(13,24,"Distribution_descriptor"),(33,43,"Distribution_descriptor"),(53,60,"Distribution_descriptor"),(88,92,"Distribution_descriptor"),(126,137,"Distribution_descriptor")]}),
# ("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor"),(5,12,"Distribution_descriptor")]}),
# ("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(54,63,"Body_size"),(64,66,"Body_size")]}),
# ("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
# ("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
# ("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]}),
# ("The marine catfish genus Paraplotosus Bleeker is reviewed. Three ",{"entities":[(4,10,"Distribution_descriptor")]})]


evaluation_data=[("Tamaki, 1984a) but it also deposit-feeds. ",{"entities":[]}),
("suspension-feeder that catches suspended food including benthic larvae (Levin, 198 1; ",{"entities":[(64,70,"Life_stage"),(56,63,"Distribution_descriptor")]}),
("makes a dwelling tube that protrudes above the sediment surface. It is basically a ",{"entities":[(47,55,"Distribution_descriptor")]}),
("length is 8 mm. This species collects detritus on the sediment surface with its palps and ",{"entities":[(0,6,"Body_size"),(10,14,"Body_size")]}),
("infaunal organism (Tamaki & Kikuchi, 1983; Tamaki, 1984a). The maximum body ",{"entities":[(0,8,"Body_size"),(63,75,"Distribution_descriptor")]}),
("sand flat. In summer, it reaches its peak numerical density and is the most dominant ",{"entities":[(0,5,"Distribution_descriptor")]}),
("This is a tube-building, spionid polychaete, and a transient member of the present ",{"entities":[]}),
("(DeWitt et al. 1990)",{"entities":[]}),
("the Antarctic Polar Front and the continental margin ",{"entities":[]}),
("D. mawsoni Norman, inhabits colder waters between ",{"entities":[(19,27,"Distribution_descriptor"),(35,41,"Distribution_descriptor")]}),
("the Antarctic Zone whereas the Antarctic toothsh, ",{"entities":[(4,18,"Distribution_descriptor")]}),
("Smitt, the Patagonian toothsh, lives primarily north of ",{"entities":[]}),
("notothenioid group dominated by small, sedentary demersal shes (Eastman 1993). Dissostichus eleginoides ",{"entities":[]}),
("and piscivorous feeding behaviour are unusual within a ",{"entities":[(24,33,"Distribution_descriptor")]}),
("Their size, neutral buoyancy, pelagic nomadic lifestyle ",{"entities":[(6,10,"Body_size")]}),
("resource and the largest fish in the Southern Ocean. ",{"entities":[(17,24,"Body_size")]}),
("toothsh of the genus Dissostichus are an important ",{"entities":[]}),
("Kock and Everson 1998), the Patagonian and Antarctic ",{"entities":[]}),
("With maximum sizes of over 100 kg (DeVries 1998; ",{"entities":[(13,18,"Body_size"),(27,33,"Body_size")]})]





development_data=[("Bleheratherina pierucciae is described from Tontouta ",{"entities":[]}),
("interations between infaunal species which can attain high densities within cages.",{"entities":[(20,28,"Distribution_descriptor")]}),
("infauna, they also provide a good opportunity to study competitive or adult-larval ",{"entities":[(0,7,"Distribution_descriptor"),(70,82,"Life_stage"),(70,75,"Life_stage"),(76,82,"Life_stage")]}),
("influence of larger predators such as fish, crabs, and epibenthic predatory benthos on ",{"entities":[(55,65,"Distribution_descriptor")]}),
("Although cages are most commonly used to assess the ",{"entities":[]}),
("with those in control plots.",{"entities":[]}),
("or in which densities of such adults are varied, and to compare the larval densities there ",{"entities":[(30,36,"Life_stage"),(68,74,"Life_stage")]}),
("establish plots from which adults which may interact with settling larvae are removed ",{"entities":[(27,33,"Life_stage"),(67,73,"Life_stage")]}),
("The basic procedure in field experiments examining adult-larval interactions is to ",{"entities":[(51,63,"Life_stage")]}),
("fin height and eye size, which are invariably greater in P. muelleri.",{"entities":[(0,10,"Body_size"),(15,23,"Body_size")]}),
("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[(13,24,"Distribution_descriptor"),(33,43,"Distribution_descriptor"),(53,60,"Distribution_descriptor"),(88,92,"Distribution_descriptor"),(126,137,"Distribution_descriptor")]}),
("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor"),(5,12,"Distribution_descriptor")]}),
("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(54,63,"Body_size"),(64,66,"Body_size")]}),
("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]}),
("The marine catfish genus Paraplotosus Bleeker is reviewed. Three ",{"entities":[(4,10,"Distribution_descriptor"),]}),("Their size, neutral buoyancy, pelagic nomadic lifestyle ",{"entities":[(6,10,"Body_size")]}),
("resource and the largest fish in the Southern Ocean. ",{"entities":[(17,24,"Body_size")]}),
("toothsh of the genus Dissostichus are an important ",{"entities":[]}),
("Kock and Everson 1998), the Patagonian and Antarctic ",{"entities":[]}),
("With maximum sizes of over 100 kg (DeVries 1998; ",{"entities":[(13,18,"Body_size"),(27,33,"Body_size")]}),
("be vulnerable to high levels of exploitation",{"entities":[]}),
("and age- or size-specific reproductive output. Current data confirm that seahorses are likely to ",{"entities":[]}),
("fishing mortalities, (c) growth rates and age at first maturity, and (d) intrinsic rates of increase ",{"entities":[]}),
("research is needed to advance seahorse conservation and management, including (a) fisheries dependent and fisheries-independent abundance estimates, (b) age- or stage-based natural and ",{"entities":[]}),
("and young increased with increasing latitude, although brood size did not. Considerable ",{"entities":[(4,9,"Life_stage")]}),
("lower than expected, perhaps because the young were more developed. The size of adults, eggs ",{"entities":[(80,86,"Life_stage"),(88,92,"Life_stage"),(72,76,"Body_size")]}),
("teleosts, even when considering only those with parental care, but brood size at release was ",{"entities":[]}),
("had no relationship to adult size. Both eggs and young were larger than expected among marine ",{"entities":[(23,28,"Life_stage"),(29,33,"Body_size"),(40,44,"Life_stage"),(49,54,"Life_stage"),(87,93,"Distribution_descriptor")]}),
("narrower size range than the 17-fold difference that occurred in adult size. Newborn body size ",{"entities":[(65,70,"Life_stage"),(71,75,"Body_size"),(65,75,"Body_size"),(85,94,"Body_size")]}),
("on species and adult size. Newborn young measured from 2 to 20 mm in  length",{"entities":[(15,20,"Life_stage"),(21,25,"Body_size"),(27,34,"Life_stage"),(55,65,"Body_size"),(69,75,"Body_size")]}),
]





In [8]:
#ksana annotate to corpus +1-2 abstracts
train_corp_bert=[("color and size of fins. Similar to other representatives of the genus Hemilepidotus, H. zapus is a benthoichthyophage with a wide food range consisting mainly of bottom and demersal crustaceans (Cirripedia, ",{"entities":[]}),
("is relatively low (on average 12800 eggs); there is dimorphism in size (females are larger than males) and in ",{"entities":[(36,40,"Life_stage")]}),
("330 g, and age reaches nine years. It spawns in August to September; the spawning is single-batch; fecundity ",{"entities":[(0,5,"Body_size")]}),
("the most short-cycled representative of the genus Hemilepidotus, whose maximum size reaches 26 cm and ",{"entities":[(71,83,"Body_size"),(92,97,"Body_size")]}),
("in the southern part (on the slopes of the underwater ridge of the outer Kurils). This species is the smallest and ",{"entities":[]}),
("the spatial-bathymetric distribution of, zapus in Pacific waters off the Northern Kurils (from 4750' to 5030' N, depths of 100-850 m) is analyzed, its thermal environment is characterized. In the northern part of the investigated area near Paramushir Island, H. zapus is represented by larger specimens staying at greater depths than ",{"entities":[(159,170,"Distribution_descriptor")]}),
("Northern Kurils from Paramushir Island to Ketoy Island. Taking into consideration trawl catches of 1996-2001, ",{"entities":[]}),
("used in the key for reliable identification of species of both genera irrespective of localities, sex, age, and condition of a specimen. In Russian waters, Hemilepidotus zapus is distributed off the Commander Islands and the ",{"entities":[(149,155,"Distribution_descriptor")]}),
("revealed. Additional taxonomic features are found. In combination with some previously used features, they are ",{"entities":[]}),
("features for the determination of taxonomical boundaries of species belonging to this group. Improved diagnoses of species are composed. Significant transgression of most features previously used in identification is ",{"entities":[]}),
("The Comparative morphological description of 248 specimens of six species of the genera Hemilepidotus and Melletes served as the basis for the assessment of the variation of over 28 external morphological ",{"entities":[]}),
("species.",{"entities":[]}),
("appears to be influenced by conflicting selection pressures that may differ between the ",{"entities":[]}),
("R. whero and competition for habitat in both species. Body size in the Ruanoho species ",{"entities":[(29,36,"Distribution_descriptor"),(54,63,"Body_size")]}),
("size differences in the Ruanoho pair are consistent with female choice for smaller males in ",{"entities":[(0,5,"Body_size")]}),
("while female R. decemdigitatus had no preference for male size. Results suggest that body ",{"entities":[(85,89,"Body_size")]}),
("in heterospecific trials. Female R. whero showed a strong preference for smaller males, ",{"entities":[]}),
("a subset of that displayed by R. decemdigitatus, while no mating behaviour was observed ",{"entities":[(55,74,"Distribution_descriptor"),(55,64,"Distribution_descriptor")]}),
("R. whero were displaced into less preferred habitats. Courtship behaviour in R. whero was ",{"entities":[(44,52,"Distribution_descriptor"),(64,73,"Distribution_descriptor")]}),
("dominant in inter- and intraspecific contests for the preferred rock habitat, while small ",{"entities":[(64,76,"Distribution_descriptor"),(64,68,"Distribution_descriptor")]}),
("trials. Both species exhibited similar habitat preferences, but large R. decemdigitatus were ",{"entities":[(39,46,"Distribution_descriptor")]}),
("setting. Reproductive behaviour was assessed in courtship, mate choice and hybridisation ",{"entities":[(9,31,"Distribution_descriptor")]}),
("trials based on the field data to test habitat preference and competitive ability in a common ",{"entities":[(39,46,"Distribution_descriptor")]}),
("substratum. The effect of body size on substratum use was investigated using laboratory ",{"entities":[(0,10,"Distribution_descriptor"),(26,35,"Body_size"),(39,49,"Distribution_descriptor")]}),
("20 m. In sites where the species co-occurred, R. whero was less associated with rock ",{"entities":[(0,4,"Distribution_descriptor")]}),
("while R. whero was frequently found in sheltered to moderately exposed areas down to ",{"entities":[(39,48,"Distribution_descriptor"),(52,70,"Distribution_descriptor")]}),
("distributions and differ in body size, attaining a total length of 12 cm and 9 cm, respectively. R. decemdigitatus was most commonly found in sheltered areas shallower than 5 m, ",{"entities":[(28,37,"Body_size"),(51,63,"Body_size"),(67,81,"Body_size"),(67,72,"Body_size"),(77,81,"Body_size"),(143,158,"Distribution_descriptor"),(159,177,"Distribution_descriptor")]}),
("New Zealand. Ruanoho decemdigitatus and Ruanoho whero have overlapping sympatric ",{"entities":[]}),
("The effect of body size on spatial resource competition and reproductive isolation was examined in a sister species pair of subtidal triplefin fishes (F. Tripterygiidae) in ",{"entities":[(14,23,"Body_size")]}),
("spawned previously. A summary of the life-cycle is also ",{"entities":[]}),
("indicated that some females in this sample had ",{"entities":[]}),
("(8.8%). A few atretic oocytes and empty follicles ",{"entities":[]}),
("stage (91.2%) or in the rst stage of vitellogenesis ",{"entities":[]}),
("and spermatozoa. Oocytes were in the previtellogenic ",{"entities":[]}),
("sample were in a resting stage. Testes lacked spermatids ",{"entities":[]}),
("histology indicated that the gonads of this November ",{"entities":[]}),
("Although specimens were large and sexually mature, the ",{"entities":[]}),
("index (IG) was 0.9% for females and 0.2% for males. ",{"entities":[]}),
("1.6% for females and 1.7% for males; gonadosomatic ",{"entities":[]}),
("relationship to W and LT. Hepatosomatic index (IH) was ",{"entities":[(16,17,"Body_size"),(22,24,"Body_size")]}),
("showed no dierence due to sex, but a signicant weak ",{"entities":[]}),
("Subcutaneous lipid thickness averaged 2.6 mm and ",{"entities":[(13,28,"Body_size"),(38,44,"Body_size")]}),
("26.7 kg (range 10.460.3 kg). The length-weight relationship was W=3.44  105 L2:85T (n 28, r2 0.96). ",{"entities":[(0,27,"Body_size"),(0,7,"Body_size")]}),
("in McMurdo Sound, Antarctica. Total length (LT) averaged 127.3 cm (range 90162 cm) and weight (W) was ",{"entities":[(30,42,"Body_size"),(44,46,"Body_size"),(30,47,"Body_size"),(57,65,"Body_size"),(67,81,"Body_size"),(57,82,"Body_size"),(87,97,"Body_size"),(87,93,"Body_size")]}),
("capture on a set line near the southern limit of its range ",{"entities":[]}),
("Twenty-eight specimens of the large notothenioid Dissostichus mawsoni were dissected after ",{"entities":[]}),
("ecological traits that are body size and trophic position.",{"entities":[(27,36,"Body_size")]}),
("predicting species loss effect on functioning based on easily measured ",{"entities":[]}),
("ecosystem functioning than random losses. Our results suggest apparent ease in ",{"entities":[]}),
("multitrophic systems, a more negative impact of ordered extinction sequences on ",{"entities":[(0,12,"Distribution_descriptor")]}),
("loss effects on functioning. Our study is the first to show experimentally, in ",{"entities":[]}),
("ordered by body size, random extinction sequences had lower multiple species ",{"entities":[(11,20,"Body_size")]}),
("effect of body size within trophic position levels. Compared to extinctions ",{"entities":[(10,19,"Body_size")]}),
("body size on functioning. The effect was much stronger when considering the ",{"entities":[(0,9,"Body_size")]}),
("and freshwater multitrophic systems. We found a significant, but weak, effect of ",{"entities":[(4,14,"Distribution_descriptor")]}),
("traits. Here, we examined the predictive capacity of species body size in marine ",{"entities":[(61,70,"Body_size"),(74,80,"Distribution_descriptor")]}),
("indicator because of its relationship to extinction risk and several functional ",{"entities":[]}),
("species loss on ecosystem functioning. Body size is often considered as a good ",{"entities":[(39,48,"Body_size")]}),
("There is an urgent need to develop predictive indicators of the effect of ",{"entities":[]}),
("lagoon population but a potential contribution to the genetic flux among coastal populations.",{"entities":[(0,6,"Distribution_descriptor"),(73,80,"Distribution_descriptor")]}),
("the estuarine larvae that drifted out of the lagoon seemed to survive, suggesting a loss of individuals from the ",{"entities":[(4,13,"Distribution_descriptor"),(14,20,"Life_stage"),(45,51,"Distribution_descriptor")]}),
("CW to LR, possibly reducing their survival. Generally, eggs were more vulnerable to drift than larvae. Many of ",{"entities":[(0,2,"Distribution_descriptor"),(6,8,"Distribution_descriptor"),(55,59,"Life_stage"),(95,101,"Life_stage")]}),
("when heavy rains led to severe lagoon discharges. Similarly, unusual strong onshore winds advected eggs from ",{"entities":[(31,37,"Distribution_descriptor"),(99,103,"Life_stage"),(69,89,"Distribution_descriptor")]}),
("larvae to LR. Eggs and larvae of estuarine species that spawned within LR were effectively retained there, except ",{"entities":[(0,6,"Life_stage"),(10,12,"Distribution_descriptor"),(14,18,"Life_stage"),(23,29,"Life_stage"),(33,42,"Distribution_descriptor"),(71,73,"Distribution_descriptor")]}),
("transport of these larvae into LR. Conversely, there was no evidence of entrance of marine estuarine-independent ",{"entities":[(19,25,"Life_stage"),(31,33,"Distribution_descriptor"),(84,90,"Distribution_descriptor"),(91,100,"Distribution_descriptor")]}),
("and light onshore winds during the open-sandbar period were the most important process that promoted the ",{"entities":[(10,23,"Distribution_descriptor")]}),
("abundance in CW. Sandbar closure did not impede the entrance of these species. Overwashing events (closed bar) ",{"entities":[(13,15,"Distribution_descriptor"),(17,24,"Distribution_descriptor"),(79,91,"Distribution_descriptor")]}),
("runoff) and species-specific traits (e.g. life cycle strategy, spawning season) determined ichthyoplankton exchange vs. retention. Marine estuarine-related species entered LR mostly during summer-autumn, at their peak ",{"entities":[(0,6,"Distribution_descriptor"),(91,106,"Distribution_descriptor"),(132,138,"Distribution_descriptor"),(139,148,"Distribution_descriptor"),(173,175,"Distribution_descriptor")]}),
("nursery ground. Larval stages of six marine species were found at CW. Environmental factors (e.g. wind and ",{"entities":[(16,22,"Life_stage"),(37,43,"Distribution_descriptor"),(66,68,"Distribution_descriptor"),(70,83,"Distribution_descriptor"),(98,102,"Distribution_descriptor")]}),
("exchange of water and ichthyoplankton. At least three estuarine species were found to spawn and use LR as a ",{"entities":[(12,17,"Distribution_descriptor"),(22,37,"Distribution_descriptor"),(54,63,"Distribution_descriptor"),(100,102,"Distribution_descriptor")]}),
("conditions and ichthyoplankton communities between LR and CW, and also evidenced different mechanisms of ",{"entities":[(51,53,"Distribution_descriptor"),(58,60,"Distribution_descriptor")]}),
("along a year in LR and CW to capture different sandbar states (open/closed) and lagoon-sea connectivity processes (e.g. lagoon discharges, marine intrusion, overwashing). Results showed differences in environmental ",{"entities":[(16,18,"Distribution_descriptor"),(23,25,"Distribution_descriptor"),(47,54,"Distribution_descriptor"),(63,74,"Distribution_descriptor"),(80,86,"Distribution_descriptor"),(87,90,"Distribution_descriptor"),(121,127,"Distribution_descriptor"),(140,146,"Distribution_descriptor"),(158,169,"Distribution_descriptor")]}),
("waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys ",{"entities":[(0,6,"Distribution_descriptor"),(8,10,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal ",{"entities":[(64,75,"Distribution_descriptor"),(76,82,"Distribution_descriptor"),(84,86,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and ",{"entities":[]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys",{"entities":[(64,87,"Distribution_descriptor"),(76,82,"Distribution_descriptor"),(101,108,"Distribution_descriptor")]}),
("and some islands in the vicinity of the latter. ",{"entities":[]}),
("known from the rivers of Australia, New Guinea ",{"entities":[(15,21,"Distribution_descriptor")]}),
("marine ancestors and until recently have only been ",{"entities":[(0,6,"Distribution_descriptor")]}),
("All freshwater atherinid species are derived from ",{"entities":[(4,14,"Distribution_descriptor")]}),
("from Tumaco, Colombia, according to Dyer. ",{"entities":[]}),
("Southeast Pacific, the family is only known only ",{"entities":[]}),
("Caribbean) known from the New World. In the ",{"entities":[]}),
("(predominantly in the shore waters of the ",{"entities":[(22,34,"Distribution_descriptor")]}),
("Atherinomorus stipes and Hypoatherina harringtonensis ",{"entities":[]}),
("World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, ",{"entities":[(17,37,"Distribution_descriptor"),(39,50,"Distribution_descriptor"),(68,74,"Distribution_descriptor")]}),
("be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old ",{"entities":[(50,59,"Body_size"),(50,56,"Body_size"),(57,59,"Body_size")]}),
("Atherinids are small marine, estuarine and freshwater fishes not exceeding 120 mm SL (a soon to ",{"entities":[(21,27,"Distribution_descriptor"),(29,38,"Distribution_descriptor"),(43,53,"Distribution_descriptor"),(15,20,"Body_size"),(75,84,"Body_size"),(82,84,"Body_size"),(75,81,"Body_size")]}),
("interations between infaunal species which can attain high densities within cages.",{"entities":[(20,28,"Distribution_descriptor")]}),
("infauna, they also provide a good opportunity to study competitive or adult-larval ",{"entities":[(0,7,"Distribution_descriptor"),(70,82,"Life_stage"),(70,75,"Life_stage"),(76,82,"Life_stage")]}),
("influence of larger predators such as fish, crabs, and epibenthic predatory benthos on ",{"entities":[(55,65,"Distribution_descriptor")]}),
("Although cages are most commonly used to assess the ",{"entities":[]}),
("with those in control plots.",{"entities":[]}),
("or in which densities of such adults are varied, and to compare the larval densities there ",{"entities":[(30,36,"Life_stage"),(68,74,"Life_stage")]}),
("establish plots from which adults which may interact with settling larvae are removed ",{"entities":[(27,33,"Life_stage"),(67,73,"Life_stage")]}),
("The basic procedure in field experiments examining adult-larval interactions is to ",{"entities":[(51,63,"Life_stage")]}),]

evaluation_data_bert=[("Tamaki, 1984a) but it also deposit-feeds. ",{"entities":[]}),
("suspension-feeder that catches suspended food including benthic larvae (Levin, 198 1; ",{"entities":[(64,70,"Life_stage"),(56,63,"Distribution_descriptor")]}),
("makes a dwelling tube that protrudes above the sediment surface. It is basically a ",{"entities":[(47,55,"Distribution_descriptor")]}),
("length is 8 mm. This species collects detritus on the sediment surface with its palps and ",{"entities":[(0,6,"Body_size"),(10,14,"Body_size")]}),
("infaunal organism (Tamaki & Kikuchi, 1983; Tamaki, 1984a). The maximum body ",{"entities":[(0,8,"Body_size"),(63,75,"Distribution_descriptor")]}),
("sand flat. In summer, it reaches its peak numerical density and is the most dominant ",{"entities":[(0,5,"Distribution_descriptor")]}),
("This is a tube-building, spionid polychaete, and a transient member of the present ",{"entities":[]}),
("conditions and plankton production show higher inter-annual variability.",{"entities":[]}),
("within relatively fixed time-frames along the year cycle, hydrographic ",{"entities":[]}),
("larval survival, and they generally assume that while spawning occurs ",{"entities":[(0,6,"Life_stage")]}),
("population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of ",{"entities":[]}),
("that survival during these stages often regulates recruitment and adult ",{"entities":[(66,71,"Life_stage")]}),
("starvation and environmental variability, and it was early recognized ",{"entities":[(15,40,"Distribution_descriptor")]}),
("early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like ",{"entities":[(0,12,"Life_stage")]}),
("Most marine fish and invertebrate species produce free and small ",{"entities":[(5,11,"Distribution_descriptor"),(59,64,"Body_size")]}), 
("molluscs and crustaceans.",{"entities":[]}),
("active nocturnally, and are known to feed on small invertebrates, particularly gastropod ",{"entities":[]}),
("substrata containing substantial amounts of sand, silt, mud, or algae. Presumably they are ",{"entities":[(44,48,"Distribution_descriptor"),(50,54,"Distribution_descriptor"),(56,59,"Distribution_descriptor"),(64,69,"Distribution_descriptor")]}),
("mainly sedentary, frequently seen resting on the bottom under rock or coral outcrops on ",{"entities":[(62,66,"Distribution_descriptor"),(70,75,"Distribution_descriptor")]}),
("of shallow reef areas, usually encountered in less than 10 m depth. During the day they are ",{"entities":[(3,21,"Distribution_descriptor")]}),
("The three species exhibit similar behaviour and ecological preferences. They are inhabitants ",{"entities":[]}),]

development_data_bert=[("fin height and eye size, which are invariably greater in P. muelleri.",{"entities":[(0,10,"Body_size"),(15,23,"Body_size")]}),
("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[(13,24,"Distribution_descriptor"),(33,43,"Distribution_descriptor"),(53,60,"Distribution_descriptor"),(88,92,"Distribution_descriptor"),(126,137,"Distribution_descriptor")]}),
("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor"),(5,12,"Distribution_descriptor")]}),
("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(54,63,"Body_size"),(64,66,"Body_size")]}),
("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]}),
("The marine catfish genus Paraplotosus Bleeker is reviewed. Three ",{"entities":[(4,10,"Distribution_descriptor"),]}),
("(DeWitt et al. 1990)",{"entities":[]}),
("the Antarctic Polar Front and the continental margin ",{"entities":[]}),
("D. mawsoni Norman, inhabits colder waters between ",{"entities":[(19,27,"Distribution_descriptor"),(35,41,"Distribution_descriptor")]}),
("the Antarctic Zone whereas the Antarctic toothsh, ",{"entities":[(4,18,"Distribution_descriptor")]}),
("Smitt, the Patagonian toothsh, lives primarily north of ",{"entities":[]}),
("notothenioid group dominated by small, sedentary demersal shes (Eastman 1993). Dissostichus eleginoides ",{"entities":[]}),
("and piscivorous feeding behaviour are unusual within a ",{"entities":[(24,33,"Distribution_descriptor")]}),
("Their size, neutral buoyancy, pelagic nomadic lifestyle ",{"entities":[(6,10,"Body_size")]}),
("resource and the largest fish in the Southern Ocean. ",{"entities":[(17,24,"Body_size")]}),
("toothsh of the genus Dissostichus are an important ",{"entities":[]}),
("Kock and Everson 1998), the Patagonian and Antarctic ",{"entities":[]}),
("With maximum sizes of over 100 kg (DeVries 1998; ",{"entities":[(13,18,"Body_size"),(27,33,"Body_size")]}),
("be vulnerable to high levels of exploitation",{"entities":[]}),
("and age- or size-specific reproductive output. Current data confirm that seahorses are likely to ",{"entities":[]}),
("fishing mortalities, (c) growth rates and age at first maturity, and (d) intrinsic rates of increase ",{"entities":[]}),
("research is needed to advance seahorse conservation and management, including (a) fisheries dependent and fisheries-independent abundance estimates, (b) age- or stage-based natural and ",{"entities":[]}),
("and young increased with increasing latitude, although brood size did not. Considerable ",{"entities":[(4,9,"Life_stage")]}),
("lower than expected, perhaps because the young were more developed. The size of adults, eggs ",{"entities":[(80,86,"Life_stage"),(88,92,"Life_stage"),(72,76,"Body_size")]}),
("teleosts, even when considering only those with parental care, but brood size at release was ",{"entities":[]}),
("had no relationship to adult size. Both eggs and young were larger than expected among marine ",{"entities":[(23,28,"Life_stage"),(29,33,"Body_size"),(40,44,"Life_stage"),(49,54,"Life_stage"),(87,93,"Distribution_descriptor")]}),
("narrower size range than the 17-fold difference that occurred in adult size. Newborn body size ",{"entities":[(65,70,"Life_stage"),(71,75,"Body_size"),(65,75,"Body_size"),(85,94,"Body_size")]}),
("on species and adult size. Newborn young measured from 2 to 20 mm in  length",{"entities":[(15,20,"Life_stage"),(21,25,"Body_size"),(27,34,"Life_stage"),(55,65,"Body_size"),(69,75,"Body_size")]}),
("fauna which invaded the freshwaters of New Caledonia.",{"entities":[(0,5,"Distribution_descriptor"),(24,35,"Distribution_descriptor")]}),
("Palaeocene, must have led to a divergence of the ancestral ",{"entities":[]}),
("Australia and its emergence as a separate island, post ",{"entities":[]}),
("events, which led to the separation of New Caledonia from ",{"entities":[]}),
("marine environment i.e. Arafura Sea. The zoogeographic ",{"entities":[(0,6,"Distribution_descriptor"),(24,35,"Distribution_descriptor")]}),
("from a common ancestor likely to have occurred in a ",{"entities":[]}),
("coastal and marine fishes, having probably been derived ",{"entities":[(0,7,"Distribution_descriptor"),(12,18,"Distribution_descriptor")]}),
("new species must have commonality with the Australian ",{"entities":[]}),
("merits a subfamilial status. The evolutionary history of this ",{"entities":[]}),
("known atherinids. Bleheratherina pierucciae can be immediately recognised by the unusual structure of its mouthparts. Other distinct osteological characters confirm that it ",{"entities":[]}),
("list since the new species is distinct and different from all ",{"entities":[]}),
("fourth subfamily, Bleheratherininae, is now added to this ",{"entities":[]}),
("into three subfamilies has been briefly reviewed and a ",{"entities":[]}),
("Leptatherina and Stenatherina) and an atherionid (Atherion). Dyer & Chernoffs (1996) division of Atherinidae ",{"entities":[]}),
("Atherion, Craterocephalus, Hypoatherina, Kestratherina, ",{"entities":[]}),
("(2656.9S 16614E) and Pirogues Rivers, New Caledonia. The new species has been compared with other IndoPacific atherinids, both freshwater and marine (representatives of genera Atherinason, Atherinomorus, Atherinosoma, ",{"entities":[(30,36,"Distribution_descriptor"),(129,139,"Distribution_descriptor"),(144,150,"Distribution_descriptor")]}),
("Bleheratherina pierucciae is described from Tontouta ",{"entities":[]})
]

In [9]:
#in order to create training data,custom ner model

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe('ner')

ner.add_label("Distribution_descriptor")
ner.add_label("Life_stage")
ner.add_label("Body_size")
ner.add_label("Sampling_device")
ner.labels


('Body_size', 'Distribution_descriptor', 'Life_stage')

In [68]:
from spacy.util import filter_spans

def training_data_format(train_data,name):
    doc_bin = DocBin()
    for training_example in tqdm(train_data): 
        text = training_example[0]
        print(text)
        labels = training_example[1]['entities']
        print(training_example[1])
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in labels:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents 
        doc_bin.add(doc)
    
    return(doc_bin.to_disk("train_" + name + ".spacy"))

# training_data_format(TRAIN_DATA,"corpus")
training_data_format(train_corp,"corp2")
training_data_format(development_data,"dev")
training_data_format(evaluation_data,"evaluation")
training_data_format(train_corp_bert,"corpus2bert")
training_data_format(evaluation_data_bert,"evalbert")
training_data_format(development_data_bert,"devbert")

# training_data_format(TRAIN_DATA_paper,"paper")

# for training_example in tqdm(evaluation_data):
#     text = training_example[0]
#     labels = training_example[1]['entities']
#     doc = nlp.make_doc(text) 
#     ents = []
#     for start, end, label in labels:
#         span = doc.char_span(start, end, label=label, alignment_mode="contract")
#         if span is None:
#             print("Skipping entity")
#         else:
#             ents.append(span)
#     filtered_ents = filter_spans(ents)
#     doc.ents = filtered_ents 
#     doc_bin.add(doc)


# doc_bin.to_disk("evaluation.spacy")

100%|██████████| 119/119 [00:00<00:00, 9173.19it/s]


Pairwise comparisons of sister families do not reveal any trends between familial diversity and any of the life history traits. The great variability of life history traits within polychaete families may explain the lack of significant results; perhaps no trends are seen because polychaete life history traits cannot be generalized at the family level.
{'entities': []}
or with planktotrophic larval development. Each of these factors has been advanced as cause for high diversity in other taxonomic assemblages. Here, we establish that the diversification pattern of polychaete families is non-random, but the data collected show no significant correlations between familial diversity and several life history traits including age at first reproduction, life span, body size, fecundity, and egg size. 
{'entities': [(23, 29, 'Life_stage')]}
Polychaetes comprise 9000 described species classified in over 70 families and exhibit a great variety of life history strategies. Our goal was to test wheth

100%|██████████| 33/33 [00:00<00:00, 8252.57it/s]


Bleheratherina pierucciae is described from Tontouta 
{'entities': []}
interations between infaunal species which can attain high densities within cages.
{'entities': [(20, 28, 'Distribution_descriptor')]}
infauna, they also provide a good opportunity to study competitive or adult-larval 
{'entities': [(0, 7, 'Distribution_descriptor'), (70, 82, 'Life_stage'), (70, 75, 'Life_stage'), (76, 82, 'Life_stage')]}
influence of larger predators such as fish, crabs, and epibenthic predatory benthos on 
{'entities': [(55, 65, 'Distribution_descriptor')]}
Although cages are most commonly used to assess the 
{'entities': []}
with those in control plots.
{'entities': []}
or in which densities of such adults are varied, and to compare the larval densities there 
{'entities': [(30, 36, 'Life_stage'), (68, 74, 'Life_stage')]}
establish plots from which adults which may interact with settling larvae are removed 
{'entities': [(27, 33, 'Life_stage'), (67, 73, 'Life_stage')]}
The basic procedure in fiel

100%|██████████| 19/19 [00:00<00:00, 6328.26it/s]


...

color and size of fins. Similar to other representatives of the genus Hemilepidotus, H. zapus is a benthoichthyophage with a wide food range consisting mainly of bottom and demersal crustaceans (Cirripedia, 
{'entities': []}
is relatively low (on average 12800 eggs); there is dimorphism in size (females are larger than males) and in 
{'entities': [(36, 40, 'Life_stage')]}
330 g, and age reaches nine years. It spawns in August to September; the spawning is single-batch; fecundity 
{'entities': [(0, 5, 'Body_size')]}
the most short-cycled representative of the genus Hemilepidotus, whose maximum size reaches 26 cm and 
{'entities': [(71, 83, 'Body_size'), (92, 97, 'Body_size')]}
in the southern part (on the slopes of the underwater ridge of the outer Kurils). This species is the smallest and 
{'entities': []}
the spatial-bathymetric distribution of, zapus in Pacific waters off the Northern Kurils (from 4750' to 5030' N, depths of 100-850 m) is analyzed, its thermal environment is charact

100%|██████████| 21/21 [00:00<00:00, 10509.53it/s]


Tamaki, 1984a) but it also deposit-feeds. 
{'entities': []}
suspension-feeder that catches suspended food including benthic larvae (Levin, 198 1; 
{'entities': [(64, 70, 'Life_stage'), (56, 63, 'Distribution_descriptor')]}
makes a dwelling tube that protrudes above the sediment surface. It is basically a 
{'entities': [(47, 55, 'Distribution_descriptor')]}
length is 8 mm. This species collects detritus on the sediment surface with its palps and 
{'entities': [(0, 6, 'Body_size'), (10, 14, 'Body_size')]}
infaunal organism (Tamaki & Kikuchi, 1983; Tamaki, 1984a). The maximum body 
{'entities': [(0, 8, 'Body_size'), (63, 75, 'Distribution_descriptor')]}
sand flat. In summer, it reaches its peak numerical density and is the most dominant 
{'entities': [(0, 5, 'Distribution_descriptor')]}
This is a tube-building, spionid polychaete, and a transient member of the present 
{'entities': []}
conditions and plankton production show higher inter-annual variability.
{'entities': []}
within relativ

100%|██████████| 48/48 [00:00<00:00, 9600.24it/s]

fin height and eye size, which are invariably greater in P. muelleri.
{'entities': [(0, 10, 'Body_size'), (15, 23, 'Body_size')]}
albilabris are similar in appearance and habits, but are separable on the basis of dorsal 
{'entities': [(41, 47, 'Distribution_descriptor')]}
by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. 
{'entities': [(13, 24, 'Distribution_descriptor'), (33, 43, 'Distribution_descriptor'), (53, 60, 'Distribution_descriptor'), (88, 92, 'Distribution_descriptor'), (126, 137, 'Distribution_descriptor')]}
Skipping entity
from coastal seas of northwestern Australia. It is clearly distinguishable from congeners 
{'entities': [(5, 17, 'Distribution_descriptor'), (5, 12, 'Distribution_descriptor')]}
butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly 
{'entities': [(54, 66, 'Body_size'), (54, 63, 'Body_size'), (64, 66, 'Body_s

In [5]:
#configuration file initiallization
!python -m spacy init fill-config base_config.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy debug data config.cfg


============================ Data file validation ============================


Traceback (most recent call last):
  File "c:\Users\melina\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\melina\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\melina\anaconda3\lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\Users\melina\anaconda3\lib\site-packages\spacy\cli\_util.py", line 71, in setup_cli
    command(prog_name=COMMAND)
  File "c:\Users\melina\anaconda3\lib\site-packages\click\core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "c:\Users\melina\anaconda3\lib\site-packages\click\core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "c:\Users\melina\anaconda3\lib\site-packages\click\core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "c:\Users\melina\anaconda3\lib\site-packages\click\core.py", line 1659, in invoke
    return _process_res

In [16]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' #fixing error
!python -m spacy train config.cfg --output ./ --paths.train ./train_corp2.spacy --paths.dev ./train_dev.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.86    0.00    0.00    0.00    0.00
  9     200         30.19   1452.54   36.62   40.62   33.33    0.37
 21     400         26.05    312.72   38.81   46.43   33.33    0.39
 35     600         28.20    189.70   36.36   44.44   30.77    0.36
 53     800         34.28    184.98   40.63   52.00   33.33    0.41
 76    1000         34.64    198.88   38.10   50.00   30.77    0.38
103    1200         53.89    228.37   38.10   50.00   30.77    0.38
135    1400         37.17    242.00   37.50   48.00   30.77    0.38
175    1600         39.28    243.43 

[2022-12-15 15:34:49,171] [INFO] Set up nlp object from config
[2022-12-15 15:34:49,180] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-12-15 15:34:49,183] [INFO] Created vocabulary
[2022-12-15 15:34:50,455] [INFO] Added vectors: en_core_web_lg
[2022-12-15 15:34:51,592] [INFO] Finished initializing nlp object
[2022-12-15 15:34:52,019] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [22]:
#corpus prwto annotation
nlp_ner_corpus2_dev=spacy.load("model-corpus2_dev")
nlp_ner_corpus2_dev.pipe_names


nlp_ner_corpusIOB_dev=spacy.load("model-corpusIOB_dev")
nlp_ner_corpusIOB_dev.pipe_names


nlp_ner_corpusIOB_dev_ruler=spacy.load("model-corpusIOB_dev_ruler")
nlp_ner_corpusIOB_dev_ruler.pipe_names


# #corpus deutero annotation
# nlp_ner_corpus2=spacy.load("model-corpus2")
# nlp_ner_corpus2.pipe_names

nlp_ner_corpus_dev=spacy.load("model-corpus_dev")
nlp_ner_corpus_dev.pipe_names

# #mono paper
# nlp_ner_paper=spacy.load("ner_paper")
# nlp_ner_paper.pipe_names

# #corpus + paper
# nlp_ner_corp_paper=spacy.load("model-corpus+paper")
# nlp_ner_corp_paper.pipe_names

['tok2vec', 'ner']

In [16]:
from spacy import displacy

doc=nlp_ner_corpus_dev("Atherinids are small marine estuarine and freshwaters fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")
doc2=nlp_ner_corpus2_dev("Atherinids are small marine estuarine and freshwaters fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")

#paper=nlp_ner_corpus("Woodin (1976) highlighted the importance of adult-larval interactions in infaunal organisms, i.e. interactions between established individuals and settling and/or settled larvae, in that such interactions could generate in a shorter time and maintain more effectively definite species assemblages as compared with adult-adult interactions.")
# paper2=nlp_ner_corpus2("Woodin (1976) highlighted the importance of adult-larval interactions in infaunal organisms, i.e. interactions between established individuals and settling and/or settled larvae, in that such interactions could generate in a shorter time and maintain more effectively definite species assemblages as compared with adult-adult interactions.")
paper2=nlp_ner_corpusIOB_dev("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc.ents])
print("\n")
print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper2.ents])

displacy.render(doc, style="ent" )
displacy.render(paper2, style="ent" )

# doc1=nlp_ner_paper("Atherinids are small marine estuarine and freshwaters fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")
# doc2=nlp_ner_corp_paper("Atherinids are small marine estuarine and freshwaters fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")

# paper1=nlp_ner_paper("Woodin (1976) highlighted the importance of adult-larval interactions in infaunal organisms, i.e. interactions between established individuals and settling and/or settled larvae, in that such interactions could generate in a shorter time and maintain more effectively definite species assemblages as compared with adult-adult interactions.")
# paper2=nlp_ner_corp_paper("Woodin (1976) highlighted the importance of adult-larval interactions in infaunal organisms, i.e. interactions between established individuals and settling and/or settled larvae, in that such interactions could generate in a shorter time and maintain more effectively definite species assemblages as compared with adult-adult interactions.")

# print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper1.ents])
# print("\n")
# print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper2.ents])

# colors = {"Distribution_descriptor": "#660066", "Body_size": "#9999FF", "Life_stage": "#99CCFF"}
# options = {"ents": ["Distribution_desciptor", "Body_size", "Life_stage"], "colors": colors}

# displacy.render(paper1, style="ent" )
# displacy.render(paper2, style="ent" )



[('marine', 'Distribution_descriptor', 21, 27, ''), ('estuarine and', 'Distribution_descriptor', 28, 41, ''), ('freshwaters', 'Distribution_descriptor', 42, 53, ''), ('150 mm', 'Body_size', 75, 81, ''), ('SL', 'Body_size', 82, 84, ''), ('300 mm SL', 'Body_size', 146, 155, ''), ('Alepidomus evermanni', 'Distribution_descriptor', 210, 230, ''), ('freshwaters', 'Distribution_descriptor', 232, 243, ''), ('marine', 'Distribution_descriptor', 261, 267, ''), ('shore waters', 'Distribution_descriptor', 352, 364, ''), ('freshwater', 'Distribution_descriptor', 511, 521, ''), ('rivers', 'Distribution_descriptor', 623, 629, '')]


[('marine', 'Distribution_descriptor', 5, 11, ''), ('adult', 'Life_stage', 339, 344, ''), ('larval', 'Life_stage', 480, 486, '')]


In [15]:
from spacy.tokenizer import Tokenizer
tokenizer = nlp.tokenizer

#in order to put the entity_ruler into nlp_ner pipeline
#entity ruler's patterns(entities) evaluate only if they are not annotaded in training data, so the ”entity_ruler” will only add new entities that match to the patterns only if they don’t overlap with existing entities predicted by the statistical model
#QUEST PWS THA EPISTREFOUME TO LINK AN EINAI APO TRAINING ?PROS TO PARON TO EPISTREFOUME APO TO ENTITY RULER mono se auta pou den exoun oristei apo training loipon
def entity_ruler(nlp_model,model):
    #nlp_ner_corpus2.remove_pipe("entity_ruler")
    if "entity_ruler" not in nlp_model.pipe_names:
        ruler=nlp_model.add_pipe("entity_ruler")
    else:
        ruler=nlp_model.get_pipe("entity_ruler")

    #GIA PATTERNS
    def dict_func(df,linkdf,label):
        patterns=[]
        j=0
        for i in df:
            dict={"label": label}
            dict["pattern"]=[{"LOWER" : str.lower(i)}]
            dict["id"]=linkdf[[1]][1][j]
            patterns.append(dict)
            tokens=tokenizer(i)
            if len(tokens) == 2:
                dict={"label": label}
                dict["pattern"]=[{"LOWER" : str.lower(str(tokens[0]))}, {"IS_PUNCT": True}, {"LOWER" : str.lower(str(tokens[1]))}]
                dict["id"]=linkdf[[1]][1][j]
                patterns.append(dict)
            j=j+1
        ruler.add_patterns(patterns)

    dict_func(df_distr_descr,df,"Distribution_descriptor")
    dict_func(df_life_stages,df1,"Life_stage")
    dict_func(df_body_size,df2,"Body_size")
    #dict_func(df_biol_descr,"Biological Descriptor")

    #in order to put entity ruler into the trained model pipeline
    nlp_model.to_disk(model)

# nlp_ner_corpusIOB_dev_ruler=spacy.load("model-corpusIOB_dev_ruler")
# nlp_ner_corpusIOB_dev_ruler.remove_pipe("entity_ruler")
# entity_ruler(nlp_ner_corpusIOB_dev_ruler,"model-corpusIOB_dev_ruler")
# nlp_ner_corpus2_dev_ruler=spacy.load("model-corpus2_dev_ruler")
# nlp_ner_corpus2_dev_ruler.remove_pipe("entity_ruler")
# entity_ruler(nlp_ner_corpus2_dev_ruler,"model-corpus2_dev_ruler")
nlp_roberta=spacy.load("model-roberta_dev")
entity_ruler(nlp_roberta,"model-roberta_dev_ruler")



In [9]:

!python -m spacy evaluate model-corpusIOB_dev_ruler/ ecology.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   0.00  
NER R   0.00  
NER F   0.00  
SPEED   5423  


=============================== NER (per type) ===============================

                             P      R      F
Distribution_descriptor   0.00   0.00   0.00
Body_size                 0.00   0.00   0.00
Life_stage                0.00   0.00   0.00
DISTRIBUTION_DESCRIPTOR   0.00   0.00   0.00
SAMPLING_METHOD           0.00   0.00   0.00
LIFE_STAGE                0.00   0.00   0.00
BODY_SIZE                 0.00   0.00   0.00



In [31]:
nlp_ner_corpusIOB_dev=spacy.load("model-corpusIOB_dev")
nlp_ner_corpusIOB_dev.pipe_names


nlp_ner_corpusIOB_dev_ruler=spacy.load("model-corpusIOB_dev_ruler")
nlp_ner_corpusIOB_dev_ruler.pipe_names

paper2=nlp_ner_corpusIOB_dev("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")
doc=nlp_ner_corpusIOB_dev_ruler("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc.ents])
print("\n")
print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper2.ents])

displacy.render(doc, style="ent" )
displacy.render(paper2, style="ent" )
nlp_ner_corpusIOB_dev_ruler.pipe_names

[('marine', 'Distribution_descriptor', 5, 11, ''), ('plankton', 'Distribution_descriptor', 100, 108, 'https://www.marinespecies.org/traits/wiki/Traits:Plankton'), ('adult', 'Life_stage', 339, 344, ''), ('larval', 'Life_stage', 480, 486, ''), ('plankton', 'Distribution_descriptor', 636, 644, 'https://www.marinespecies.org/traits/wiki/Traits:Plankton')]


[('marine', 'Distribution_descriptor', 5, 11, ''), ('adult', 'Life_stage', 339, 344, ''), ('larval', 'Life_stage', 480, 486, '')]


['tok2vec', 'ner', 'entity_ruler']

In [63]:
from spacy import displacy

combined_model=spacy.load("combined_IOBspacy_roberta_dev_ruler")
combined_model.to_disk("combined_IOBspacy_roberta_dev_ruler")


In [60]:
#combining 2 ner from trained models (roberta and spacy model)

# nlp_roberta=spacy.load("model-roberta_dev_ruler")
# nlp_roberta.rename_pipe("ner", "ner_roberta")
# nlp_roberta.to_disk("model-roberta_dev_ruler")

nlp1 = spacy.load("model-corpusIOB_dev_ruler")

# Load the second model
nlp2 = spacy.load("model-roberta_dev_ruler")
# Add the components from the second model to the first
for name, component in nlp2.pipeline:
    if name  not in nlp1.pipe_names:
        nlp1.add_pipe(name=name,source=nlp2,factory_name=name)
    
nlp1.to_disk("combined_IOBspacy_roberta_dev_ruler")

nlp1=spacy.load("combined_IOBspacy_roberta_dev_ruler")
doc = nlp1("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")

displacy.render(doc, style="ent" )

c:\Users\melina\anaconda3\lib\site-packages\spacy\language.py:716: UserWarning: [W113] Sourced component 'transformer' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))


In [ ]:
#TESTING AND NOTES BELOW
# nlp_ner_corpus2.to_disk("model-corpus2")
# nlp_ner_corpus2=spacy.load("model-corpus2")

# nlp_ner_corpus2.pipe_names

#nlp_ner_corpus2.remove_pipe("entity_ruler")

#example of fixing dictionary patterns
# patterns = [{"label": "Distribution_descriptor", "pattern": [{"LOWER": "beta"}, {"IS_PUNCT": True},{"LOWER": "mesohaline"}]}]
# ruler.add_patterns(patterns)
# nlp_ner_corpus2.to_disk("model-corpus2")
# doc=nlp_ner_corpus2("Beta mesohaline")

# print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc.ents])


#psaxnw gia to pws tha valoume na epistrefei link me plhroforia
# nlp_ner.add_pipe("entity_linker", last=True)

# all_linked_entities = paper._.linkedEntities
# # iterates over sentences and prints linked entities
# for sent in doc.sents:
#     sent._.linkedEntities.pretty_print()

#eipame mhpws kapws meta to training pairnei ta text twn ents kai ta linkaroume kai ta epistrefoume,alliws sta leksika kai epistrefetai link gia o,ti exoume entry sto leksiko

In [ ]:
#gia na dw kapws an mpainei tagger kai parser sto pipeline
# nlp_ner_corpus.remove_pipe("tagger")
# #nlp_ner_corpus.add_pipe("tagger", before="ner")
# #nlp_ner_corpus.add_pipe("parser", after="tagger")
# nlp_ner_corpus.add_pipe("tagger", source=spacy.load("en_core_web_sm"))


In [ ]:
#prosptatheia metatrophs spacy annotations to iob

# from spacy.training import offsets_to_biluo_tags
# from spacy.training import biluo_tags_to_spans
# TRAIN_DATA = [
#     ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
#     ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
# ]

# nlp = spacy.load('en_core_web_sm')
# docs = []
# for text, annot in TRAIN_DATA:
#     doc = nlp(text)
#     tags = offsets_to_biluo_tags(doc, annot['entities'])
#     entities = biluo_tags_to_spans(doc, tags)
#     doc.ents = entities
#     docs.append(doc)
#     # then convert L->I and U->B to have IOB tags for the tokens in the doc